### Установка и импорт всех необходимых зависимостей

In [ ]:
!pip install -q razdel
!pip install -q pymorphy2
!pip install -q git+https://github.com/ahmados/rusynonyms.git
!pip install -q natasha

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

import nltk
from nltk.corpus import stopwords
import re
import pymorphy2
from razdel import tokenize
from razdel import sentenize
import string
from natasha import (
    MorphVocab,
    NewsMorphTagger,
    NewsEmbedding,
    Segmenter,
    NewsSyntaxParser,
    Doc
)

import torch
import tensorflow_hub as hub
from torch import nn
from torch.utils.data import Dataset, DataLoader
import transformers
import numpy as np

from tqdm import tqdm
import os
import sys
from typing import *

from lime.lime_text import LimeTextExplainer
import shap

nltk.download('stopwords')
nltk.download('punkt')
rus_stopwords = stopwords.words('russian')
punctuation = list(string.punctuation)

### Работа с данными (kaggle)

In [ ]:
datasets_folder = '/kaggle/input/sw-datasets/Russian-Sentiment-Analysis-Evaluation-Datasets'
datasets = ['SentiRuEval-2015-telecoms', 'SentiRuEval-2015-banks', 'SentiRuEval-2016-banks', 'SentiRuEval-2016-telecoms']
samples = ['test.xml', 'train.xml', 'test_etalon.xml']

In [ ]:
def extract_data(path: str) -> pd.DataFrame:
    """
    функция для извлечения данных из xml
    """
    tree = ET.parse(path)
    root = tree.getroot()
    DataFrame = dict()
    database = root.findall('database')[0]
    DataFrame_columns = list()

    for idx, table in enumerate(database.findall('table')):
        for column in table.findall('column'):
            DataFrame[column.attrib['name']] = list()
            DataFrame_columns.append(column.attrib['name'])
        if idx == 0:
            break

    for table in database.findall('table'):
        for column in table.findall('column'):
            DataFrame[column.attrib['name']].append(column.text)

    data = pd.DataFrame(DataFrame, columns=DataFrame_columns)
    return data

# инициализация всех путей (kaggle)
banks_dataset = datasets[2]
path2samples = os.path.join(datasets_folder, banks_dataset)
banks = ['sberbank', 'vtb', 'gazprom', 'alfabank', 'bankmoskvy', 'raiffeisen', 'uralsib', 'rshb']

path2test = os.path.join(path2samples, samples[2])
data_test = extract_data(path2test)

path2train = os.path.join(path2samples, samples[1])
data_train = extract_data(path2train)

In [ ]:
def extract_text_features(data: pd.DataFrame) -> pd.DataFrame:
    """
    функция для первичной обработки текста от лишних символов
    """
    extracted_data = dict()
    extracted_data['text'] = list()
    extracted_data['0class'] = list()
    extracted_data['1class'] = list()

    for idx in range(len(data)):
        row = data.iloc[idx, :]
        banks_review = row[banks]
        unique_labels = set(banks_review)
        unique_labels.remove('NULL')

        # убираем все ненужные знаки
        filtered_text = re.sub('http[A-z|:|.|/|0-9]*', '', row['text']).strip()
        filtered_text = re.sub('@\S*', '', filtered_text).strip()
        filtered_text = re.sub('#', '', filtered_text).strip()
        new_text = filtered_text

        # сохраняем только уникальные токены (без придатка xml NULL)
        unique_labels = list(unique_labels)
        while len(unique_labels) < 2:
            unique_labels.append(unique_labels[-1])
        extracted_data['text'].append(new_text)
        for idx, label in enumerate(unique_labels):
            text_label = int(label) + 1
            extracted_data[f'{idx}' + 'class'].append(text_label)

    extracted_data = pd.DataFrame(extracted_data)
    
    # возвращаем dataframe
    return extracted_data

extracted_test = extract_text_features(data_test)
extracted_train = extract_text_features(data_train)

In [ ]:
# пример твита из датасета
extracted_test.iloc[3308].text

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# анализ распределения таргетов на твитах
fig, axes = plt.subplots(1, 2, figsize=(8, 5))
plt.subplots_adjust(hspace=0.15, wspace=0.3)

graph1 = sns.countplot(data=extracted_train, x='0class', ax=axes[0])
graph1.set(xlabel='class_num', ylabel='amount of class', title='Amount of classes according 1 label')
graph1.grid(True)

graph2 = sns.countplot(data=extracted_train, x='1class', ax=axes[1])
graph2.set(xlabel='class_num', ylabel='amount of class', title='Amount of classes according 2 label')
graph2.grid(True)

None

### Инициализируем модель (fine-tune) для решения нашей задачи классификации

In [ ]:
learning_rate = 1e-05


class BERTmy(torch.nn.Module):
    def __init__(self, n_classes: int) -> None:
        super(BERTmy, self).__init__()
        self.rubert = transformers.AutoModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence"
        )
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", 
            do_lower_case=True,
            add_additional_tokens=True
        )
        
        hidden_size_output = self.rubert.config.hidden_size
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_output, hidden_size_output, bias=True),
            torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size_output, n_classes),
        )

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, 
        token_type_ids: torch.Tensor, output_attentions: bool=False
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        rubert_output = self.rubert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
            output_attentions=output_attentions
        )
        if not output_attentions:
            pooled = rubert_output['pooler_output']
        else:
            pooled, attentions = rubert_output['pooler_output'], rubert_output['attentions']

        output = self.classifier(pooled)

        if not output_attentions:
            return output
        else:
            return output, attentions
    
    def configure_optimizer(
        self, use_scheduler: bool=False
    ) -> torch.optim:
        # freeze part of params
        encoder_size = 0
        for param in self.rubert._modules['encoder'].parameters():
            encoder_size += 1
        encoder_size_half = encoder_size // 2
        for idx, param in enumerate(self.rubert._modules['encoder'].parameters()):
            param.requires_grad = False
            if idx >= encoder_size_half:
                break
        
        # Adam
        optimizer = torch.optim.Adam(
            params=[
                {'params':self.rubert._modules['embeddings'].parameters(), 'lr':4e-6},
                {'params':self.rubert._modules['encoder'].parameters(), 'lr':4e-6},
                {'params':self.rubert._modules['pooler'].parameters(), 'lr':4e-6},
                {'params':self.classifier.parameters(), 'lr':9e-5}
            ],
            lr=learning_rate
        )
        if use_scheduler:
            # scheduler
            scheduler = torch.optim.lr_scheduler.ExponentialLR(
                optimizer, gamma=0.96
            )
        
            return optimizer, scheduler
        
        else:
            return optimizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_cls = len(pd.unique(extracted_train['0class']))
bert = BERTmy(num_cls)
if torch.cuda.is_available():
    bert = bert.cuda()
optimizer, scheduler = bert.configure_optimizer(use_scheduler=True)

### Инициализируем class для нашего датасета

In [ ]:
train_batch_size = 32
val_batch_size = 16

class SentimentData(Dataset):
    # инициализация датасета
    def __init__(
        self, dataframe: pd.DataFrame, mode: str, 
        col_name: str, split_param: float=0.9
    ) -> None:
        self.mode = mode # train/test
        self.data = dataframe # data
        self.col_name = col_name # column for analyzing
        
        data_size = self.data.shape[0]
        if self.mode in ['val', 'train']:
            if self.mode == 'train':
                self.data = self.data.iloc[:int(data_size * split_param)]
            else:
                self.data = self.data.iloc[int(data_size * split_param):]
        
        assert self.mode in ['val', 'train', 'test']

    # для получения размера датасета
    def __len__(self) -> int:
        return self.data.shape[0]

    # для получения элемента по индексу
    def __getitem__(
        self, index: int
    ) -> Dict[str, Union[str, torch.Tensor]]:
        text = self.data.iloc[index][self.col_name]
        target1 = self.data.iloc[index]['0class']
        target2 = self.data.iloc[index]['1class']

        return {
            'text': text,
            'target1': torch.tensor(target1, dtype=torch.long),
            'target2': torch.tensor(target2, dtype=torch.long)
        }

### Инициализируем наши DataLoaders

In [ ]:
train = SentimentData(
    dataframe=extracted_train,
    split_param=1.0,
    mode='train',
    col_name='text'
)

val = SentimentData(
    dataframe=extracted_train,
    mode='val',
    col_name='text'
)

test = SentimentData(
    dataframe=extracted_test,
    mode='test',
    col_name='text'
)

train_loader = DataLoader(train, batch_size=train_batch_size, shuffle=True)
# val_loader = DataLoader(val, batch_size=val_batch_size, shuffle=False)
loaders = {
    'train': train_loader,
    # 'val': val_loader
}

### Дообучение модели

In [ ]:
def train_model(
    epochs: int, model: torch.nn.Module, loaders: List[DataLoader], 
    optimizer: torch.optim, scheduler: torch.optim.lr_scheduler
) -> torch.nn.Module:
    # cross entropy loss
    loss_function1 = torch.nn.CrossEntropyLoss()
    loss_function2 = torch.nn.CrossEntropyLoss()
    rubert_tokenizer = bert.tokenizer
    
    # извлечение DataLoaders
    if len(loaders) > 1:
        train_loader = loaders['train']
        val_loader = loaders['val']
        steps_per_epoch = [('train', train_loader), ('val', val_loader)]
    else:
        train_loader = loaders['train']
        steps_per_epoch = [('train', train_loader)]

    # обучение по эпохам
    for epoch in range(epochs):
        for mode, loader in steps_per_epoch:
            # сохранение статистик
            train_loss = 0
            n_correct = 0
            processed_data = 0
            
            # train/val 
            if mode == 'train':
                model.train()
                requires_grad_mode = True
            else:
                model.eval()
                requires_grad_mode = False
            
            # проход по батчам
            for data in tqdm(loader):
                # обнуляем градиенты
                optimizer.zero_grad()

                # извлечение входных данных для модели
                inputs = rubert_tokenizer(
                    data['text'], padding=True, truncation=True, 
                    add_special_tokens=True, return_tensors='pt'
                )
                ids = inputs['input_ids'].to(device)
                mask = inputs['attention_mask'].to(device)
                token_type_ids = inputs["token_type_ids"].to(device)
                target1 = data['target1'].to(device)
                target2 = data['target2'].to(device)
                
                # устанавливаем необходимость вычислять/не_вычислять градиенты
                with torch.set_grad_enabled(requires_grad_mode):
                    outputs = model(ids, mask, token_type_ids)
                    preds = torch.argmax(outputs.data, dim=1)

                    # настраиваем модели на конкретный target
                    if all(target1 == target2):
                        loss1 = loss_function1(outputs, target1)
                        train_loss += loss1.item() * outputs.size(0)
                        n_correct += torch.sum(preds == target1)
                        if mode == 'train':
                            # вычисляем градиенты и обновляем веса
                            loss1.backward()
                            optimizer.step()
                    # если у твита более чем 1 метка, то настраиваем на обе
                    else:
                        loss1 = loss_function1(outputs, target1) * 0.5
                        loss2 = loss_function2(outputs, target2) * 0.5
                        loss_all = loss1 + loss2
                        train_loss += loss_all.item() * outputs.size(0)

                        mask_singular = target1 == target2
                        mask_multiple = target1 != target2
                        singular = preds[mask_singular]
                        n_correct += torch.sum(singular == target1[mask_singular])
                        multiple = preds[mask_multiple]
                        n_correct += torch.sum((multiple == target1[mask_multiple]) & (multiple == target2[mask_multiple]))
                        if mode == 'train':
                            # вычисляем градиенты и обновляем веса
                            loss_all.backward()
                            optimizer.step()     
                    processed_data += outputs.size(0)

            # вычисляем ошибку и точность прогноза на эпохе
            loader_loss = train_loss / processed_data
            loader_acc = n_correct.cpu().numpy() / processed_data
            print(f'{epoch + 1} epoch with {mode} mode has: {loader_loss} loss, {loader_acc} acc')
        
        # делаем шаг для sheduler оптимайзера
        scheduler.step()

    return model

In [ ]:
epochs = 12
bert = train_model(epochs, bert, loaders, optimizer, scheduler)

In [ ]:
mode_process = input('Load weights? (y/n)')
if mode_process == 'n':
    torch.save(bert.state_dict(), 'bert_weights_pooled.pth')
elif mode_process == 'y':
    bert.load_state_dict(torch.load('/kaggle/input/bert-weights-better/bert_weights_pooled.pth'))
else:
    assert mode_process in ['n', 'y']
bert.eval()
None

### Вычисление итоговых показателей

In [ ]:
def calculate_accuracy(
    model: torch.nn.Module, SentimentData:Dataset
) -> float:
    model.eval()
    loader = DataLoader(SentimentData, batch_size=10, shuffle=False)
    n_correct = 0
    processed_data = 0
    
    for data in tqdm(loader):
        inputs = model.tokenizer(
            data['text'], padding=True, 
            add_special_tokens=True, return_tensors='pt'
        )
        ids = inputs['input_ids'].to(device)
        mask = inputs['attention_mask'].to(device)
        token_type_ids = inputs["token_type_ids"].to(device)
        target1 = data['target1'].to(device)
        target2 = data['target2'].to(device)
        
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            preds = torch.argmax(outputs.data, dim=1)
            mask_singular = target1 == target2
            mask_multiple = target1 != target2
            singular = preds[mask_singular]
            n_correct += torch.sum(singular == target1[mask_singular])
            multiple = preds[mask_multiple]
            if len(multiple) > 0:
                n_correct += torch.sum((multiple == target1[mask_multiple]) & (multiple == target2[mask_multiple]))
            processed_data += outputs.size(0)
        
    loader_acc = n_correct.cpu().numpy() / processed_data
    
    return loader_acc

def calculate_f1_class(
    model: torch.nn.Module, SentimentData: Dataset, class_num: int
) -> float:
    model.eval()
    loader = DataLoader(SentimentData, batch_size=10, shuffle=False)
    true_positive = 0
    false_positive, false_negative = 0, 0
    
    for data in tqdm(loader):
        inputs = model.tokenizer(
            data['text'], padding=True, 
            add_special_tokens=True, return_tensors='pt'
        )
        ids = inputs['input_ids'].to(device)
        mask = inputs['attention_mask'].to(device)
        token_type_ids = inputs["token_type_ids"].to(device)
        target1 = data['target1'].to(device)
        target2 = data['target2'].to(device)
        
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            
            preds = torch.argmax(outputs.data, dim=1)
            preds = preds.cpu().numpy()
            target1 = target1.cpu().numpy()
            
            mask_positive = target1 == class_num
            mask_negative = target1 != class_num
            
            true_positive += np.sum(preds[mask_positive] == class_num)
            false_positive += np.sum(preds[mask_negative] == class_num)
            false_negative += np.sum(preds[mask_positive] != class_num)
        
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    loader_f1 = 2 * precision * recall / (precision + recall)
    
    return loader_f1

In [ ]:
test_acc = calculate_accuracy(bert, test)
class_neg_f1 = calculate_f1_class(bert, test, 0)
class_neu_f1 = calculate_f1_class(bert, test, 1)
class_pos_f1 = calculate_f1_class(bert, test, 2)

In [ ]:
# общая accuracy и f1 по классам
test_acc, class_neg_f1, class_neu_f1, class_pos_f1

### Backdoor attacks on neural network(adversial examples)

#### USE metric for similarity between original sentence and spoiled sentence

In [ ]:
def use_score(original, adversial, use_bert_encoder=False, model=None):
    from scipy.spatial.distance import cosine
    # Load pre-trained universal sentence encoder model
    if not use_bert_encoder:
        # using DAN from tensorflow
        use_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

        sentences_orig = list()
        sentences_adv = list()
        for pair in zip(original, adversial):
            orig, adv = pair
            sentences_orig.append(orig)
            sentences_adv.append(adv)

        # get embs of texts
        sentences_orig_emb = use_encoder(sentences_orig)
        sentences_adv_emb = use_encoder(sentences_adv)

        # calculate use_score with DAN
        use_scores = list()
        for pair in zip(sentences_orig_emb, sentences_adv_emb):
            orig_emb, adv_emb = pair[0], pair[1]
            use_score_one = 1 - cosine(orig_emb, adv_emb)
            use_scores.append(use_score_one)
    else:
        # using BERT itself
        def get_inputs(text): # get inputs for model
            inputs = model.tokenizer(
                text, padding=True, 
                add_special_tokens=True, 
                return_tensors='pt'
            )
            ids = inputs['input_ids'].type(torch.long).to(device)
            mask = inputs['attention_mask'].type(torch.long).to(device)
            token_type_ids = inputs["token_type_ids"].type(torch.long).to(device)
            
            return ids, mask, token_type_ids

        # calculate use_score with BERT
        use_scores = list()
        for pair in zip(original, adversial):
            orig, adv = pair[0], pair[1]
            orig_inputs = get_inputs(orig)
            adv_inputs = get_inputs(adv)
            orig_outputs = model.rubert(*orig_inputs)
            adv_outputs = model.rubert(*adv_inputs)
            orig_pooled, adv_pooled = orig_outputs[1], adv_outputs[1]
            orig_pooled = orig_pooled.cpu().detach().numpy()
            adv_pooled = adv_pooled.cpu().detach().numpy()
            use_score_one = 1 - cosine(orig_pooled, adv_pooled)
            use_scores.append(use_score_one)
    
    return use_scores, np.mean(use_scores)

## Attention visualization

In [ ]:
def visualize_attention_one_head(tokens, attention_weights, num_layer, num_head):
    # works only with batch_size=1
    num_layer -= 1
    num_head -= 1
    assert num_head >= 0 and num_head < len(attention_weights[0][0])
    assert num_layer < len(attention_weights) and num_layer >= 0
    
    attention_layer = attention_weights[num_layer][0].cpu().detach().numpy()
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 6))

    g = sns.heatmap(attention_layer[num_head], annot=True, linewidth=0.1, fmt='.1g')
    # xlabel='weight_for_embed', ylabel='num_embed'
    g.set(title=f'layer: {num_layer + 1}; head: {num_head + 1} attention map')
    tickvalues = range(0,len(tokens) + 2)
    tokens = ['CLS'] + tokens + ['SEP']
    g.set_yticks(ticks=tickvalues ,labels=tokens, rotation='horizontal')
    g.set_xticks(ticks=tickvalues ,labels=tokens, rotation='vertical')
    ax = g
    plt.show()

In [ ]:
adversial_examples_char = pd.read_csv('/kaggle/input/result-data/adversial_examples_char.csv')
text_example = extracted_test.iloc[10].text
text_example_ins = adversial_examples_char['1_ins_amount_1_SpoiledText'].iloc[10]
text_example_del = adversial_examples_char['1_del_amount_1_SpoiledText'].iloc[10]
text_example_sub = adversial_examples_char['1_sub_amount_1_SpoiledText'].iloc[10]

In [ ]:
def visualize_attention_for_text(text, num_layer, num_head):
    text_seq = bert.tokenizer(
        text,
        padding=True,
        add_special_tokens=True,
        return_tensors='pt'
    ).to(device)
    logits, attention = bert(**text_seq, output_attentions=True)
    tokens = bert.tokenizer.tokenize(text)
    visualize_attention_one_head(tokens, attention, num_layer, num_head)

In [ ]:
visualize_attention_for_text(text_example_sub, 12, 1)

### utils for generating adversarial text

In [ ]:
key_errors = {
    'й': ['ц', 'ы', 'ф'],
    'ц': ['й', 'ы', 'у'],
    'у': ['ц', 'в', 'к'],
    'к': ['у', 'а', 'е'],
    'е': ['к', 'п', 'н'],
    'н': ['е', 'р', 'г'],
    'г': ['н', 'о', 'ш'],
    'ш': ['г', 'л', 'щ'],
    'щ': ['ш', 'д', 'з'],
    'з': ['щ', 'ж'],
    'х': ['ъ', 'э', 'з'],
    'ъ': ['э', 'х'],
    'ф': ['й', 'ы', 'я'],
    'ы': ['ц', 'в', 'ч', 'ф'],
    'в': ['у', 'а', 'с', 'ы'],
    'а': ['к', 'п', 'м', 'в'],
    'п': ['е', 'р', 'и', 'а'],
    'р': ['н', 'о', 'т', 'п'],
    'о': ['г', 'л', 'ь', 'р'],
    'л': ['ш', 'д', 'б', 'о'],
    'д': ['щ', 'ж', 'ю', 'л', 'б'],
    'ж': ['з', 'э', 'ю', 'д'],
    'э': ['х', 'ъ', 'ж'],
    'я': ['ф', 'ы', 'ч'],
    'ч': ['ы', 'в', 'с', 'я'],
    'с': ['в', 'а', 'м', 'ч'],
    'м': ['а', 'п', 'и', 'с'],
    'и': ['п', 'р', 'т', 'м'],
    'т': ['р', 'о', 'ь', 'и'],
    'ь': ['о', 'л', 'б', 'т'],
    'б': ['ь', 'л', 'д', 'ю'],
    'ю': ['д', 'ж', 'б'],
    'r': ['t', 'f', 'e'],
    't': ['y', 'f', 'e'],
    '0': ['9', '-'],
    '1': ['`', '2'],
    '2': ['1', '3'],
    '3': ['2', '4'],
    '4': ['3', '5'],
    '5': ['4', '6'],
    '6': ['5', '7'],
    '7': ['6', '8'],
    '8': ['7', '9'],
    '9': ['8', '0'],
    '-': ['0', '+'],
    'k': ['i', 'j', 'l', 'm'],
    '.': [',', '/', 'l', ';']
}
# получаем словарь формата: буква -> ближайшие буквы на клавиатуре

### Prepare data adversarial generating

In [ ]:
# выбираем текст для генерации состязательных примеров с сохранением исходной пропорции
limit_neu = 1300
limit_pos = 270
limit_neg = 550
adversial_examples_pos = extracted_test[extracted_test['0class'] == 2]
adversial_examples_neu = extracted_test[extracted_test['0class'] == 1]
adversial_examples_neg = extracted_test[extracted_test['0class'] == 0]

adversial_examples_pos = adversial_examples_pos.head(limit_pos)
adversial_examples_neu = adversial_examples_neu.head(limit_neu)
adversial_examples_neg = adversial_examples_neg.head(limit_neg)

adversial_examples = pd.concat([adversial_examples_pos, adversial_examples_neu, adversial_examples_neg])
adversial_examples_char = adversial_examples.sample(frac=1)

print('Размер текста для генерации: ', len(adversial_examples_char))
print('Баланс классов: ')
print(np.unique(adversial_examples_char['0class'], return_counts=True))

### Work with word importance

In [ ]:
def gather_back_tokens(tokens: List[str], tokens_type: str) -> str:
    """
    для превращения токенов в предложение
    tokens: список токенов
    tokens_type: natasha или razdel
    """
    assert tokens_type in ['razdel', 'natasha']

    sent = ''
    prev_end = None
    for token in tokens:

        if tokens_type == 'natasha':
            token_text = token['text']
            token_start, token_stop = token['start'], token['stop']
        else:
            token_text = token.text
            token_start, token_stop = token.start, token.stop
        
        if not prev_end is None:
            sent += (token_start - prev_end) * ' '

        sent += token_text
        prev_end = token_stop
 
    return sent


# get inputs for model
def get_inputs(text):
    inputs = bert.tokenizer(
        text, padding=True, truncation=True, 
        add_special_tokens=True, return_tensors='pt'
    )

    ids = inputs['input_ids'].type(torch.long).to(device)
    mask = inputs['attention_mask'].type(torch.long).to(device)
    token_type_ids = inputs["token_type_ids"].type(torch.long).to(device)

    # return input for model
    return ids, mask, token_type_ids


def predict_text(text):
    """
    for Lime: return probability distribution of text
    """
    # get model outputs
    ids, mask, token_type_ids = get_inputs(text)
    with torch.no_grad():
        outputs = bert(ids, mask, token_type_ids)
    
    # get probs
    probs = torch.nn.functional.softmax(outputs, dim=1).cpu().detach().numpy()

    return probs


def RazdelSplit(text):
        
    return [raz_tok.text for raz_tok in list(tokenize(text))]

def NatashaSplit(text):
    
    segmenter = Segmenter()
    text_doc = Doc(text.lower())
    text_doc.segment(segmenter)
    
    return [nat_tok['text'] for nat_tok in text_doc]


# get words score to final output
def extract_essential_words(
    tokens: List[str], target: int, tok_imoprtance: str, 
    tokens_type: str, num_samples: int=850, num_featyres: int=150
) -> List[Tuple[str, int]]:
    """
    возвращает список слов по убыванию важности
    причем если на вход поданы токены natasha
    то вернет токены natasha
    а если на вход - токены razdel
    то вернет токены razdel
    """

    assert mode in ['loss', 'lime', 'shap']
    assert tokens_type in ['razdel', 'natasha']

    # список для наиболее важных слов
    essential_words = list()
    
    # восстанавливаем текст из слов
    text_to_explain = gather_back_tokens(tokens,tokens_type)

    if mode == 'lime':
        
        if tokens_type == 'razdel':
            Spliter = RazdelSplit
        elif token_type == 'natasha':
            Spliter = NatashaSplit
        # создаем Explainer
        explainer = LimeTextExplainer(
            class_names=['Neg', 'Neu', 'Pos'],
            split_expression=Spliter
        )

        # "объясняем" текст
        explanation = explainer.explain_instance(
            text_to_explain, predict_text, 
            num_features=num_features, num_samples=num_samples
        )

        # создаем mapping из токена в его вес LogReg
        explanation_list = explanation.as_list()
        tok2weight = {token:weight for token, weight in explanation_list}
        
        # создаем список из токенов, их важности и позиции в тексте
        for token in tokens:
            if tokens_type == 'razdel':
                token_text = token.text.lower()
            else:
                token_text = token['text'].lower()
            
            essential_words.append((
                token, tok2weight[token_text]
            ))
        
        # создаем функцию сравнения важности
        sort_func = lambda x: np.abs(x[1])
    
    elif mode == 'shap':

        def f(x):
            print(x)
            import time
            time.sleep(1)
            tv = torch.tensor(
                [
                    tokenizer.encode(v, padding="max_length", max_length=128, truncation=True)
                    for v in x
                ]
            ).cuda()
            attention_mask = (tv != 0).type(torch.int64).cuda()
            outputs = model(tv, attention_mask=attention_mask)[0].detach().cpu().numpy()
            scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
            val = sp.special.logit(scores)
            return val

        def custom_tokenizer(s, return_offsets_mapping=True):
            """Custom tokenizers conform to a subset of the transformers API."""
            pos = 0
            offset_ranges = []
            input_ids = []
            for m in re.finditer(r"\W", s):
                start, end = m.span(0)
                offset_ranges.append((pos, start))
                input_ids.append(s[pos:start])
                pos = end
            if pos != len(s):
                offset_ranges.append((pos, len(s)))
                input_ids.append(s[pos:])
            out = {}
            out["input_ids"] = input_ids
            if return_offsets_mapping:
                out["offset_mapping"] = offset_ranges
            return out

        masker = shap.maskers.Text(custom_tokenizer)
        explainer = shap.Explainer(f, masker, output_names=labels)
    
    elif mode == 'alti':
        
        pass
        
    elif mode == 'loss':
        
        loss = torch.nn.CrossEntropyLoss()
        
        # get inputs and outputs from model
        ids, mask, token_type_ids = get_inputs(text_to_explain)
        outputs = bert(ids, mask, token_type_ids)

        # calculate loss for original text
        loss_score_integral = loss(outputs.cpu(), torch.tensor([target], dtype=torch.long))

        for idx, token in enumerate(tokens):
            # get text without one token
            tokens_copy = tokens.copy()
            tokens_copy.pop(idx)
            text_to_explain = gather_back_tokens(tokens_copy, tokens_type=tokens_type)

            # calculate loss without current word
            ids, mask, token_type_ids = get_inputs(text_to_explain)
            with torch.no_grad():
                outputs = bert(ids, mask, token_type_ids)
            loss_score_part = loss(outputs.cpu(), torch.tensor([target], dtype=torch.long))
            # add our score of change
            essential_words.append((
                token, (loss_score_part - loss_score_integral).cpu().detach().numpy()
            ))
            # создаем функцию сравнения важности
            sort_func = lambda x: x[1]
    
    # сортируем токены по важности
    essential_words = sorted(essential_words, key=sort_func, reverse=True)

    # возвращаем только слова и их позиции в тексте
    essential_words = [(word, pos) for word, score, pos in essential_words]

    return essential_words


def extract_random_words(
    tokens: List[str]
) -> List[Tuple[str, int]]:
    """
    возвращает список слов в случайном порядке
    """
    permutation = np.random.permutation(len(tokens))

    return [tokens[idx] for idx in permutation]

## char-level attacks

In [ ]:
# функция для генерации состязательных примеров на уровне символов
def extract_spoiled_text_char_level(
        dataframe, words2spoil=2, 
        sub_percent=0.15, sub_amount=1,
        mode2spoil='mixed', mode2amount='percent',
        tok_importance='loss'
    ):


    def get_indexes2change(
        sub_letter: int, token_len: int
    ) -> List[int]:
        """
        функция для получения индексов букв на замену (кроме 0)
        """
        lst_to_random = list(range(1, token_len))
        np_ids = np.random.choice(lst_to_random, size=sub_letter, replace=False)

        return np_ids.tolist()
    
    def make_token_change(
        indexes: List[int], token: str, mode='mixed'
    ):
        """
        фукнция для замены букв по индексам с использованием 4 типов замены:
        del: только удаление
        ins: только вставка
        sub: только замена
        mixed: все вместе сразу
        """
        if mode == 'sub':
            # заменяем букву на позиции
            word = list(token.text)
            for idx in indexes:
                symbol = word[idx]
                try:
                    word[idx] = key_errors[symbol][random.randint(0, len(key_errors[symbol])-1)]
                except:
                    pass
            return (token.start, token.stop, ''.join(word), 0)
        elif mode == 'ins':
            # вставляем букву на позиция и увеличиваем длину токена на 1
            ins_count = 0
            word = list(token.text)
            indexes = sorted(indexes)
            for idx in indexes:
                symbol = word[idx+ins_count]
                try:
                    word.insert(idx+ins_count, key_errors[symbol][random.randint(0, len(key_errors[symbol]) - 1)])
                    ins_count += 1
                except:
                    pass
            return (token.start, token.stop, ''.join(word), ins_count)
        elif mode == 'del':
            # удаляем букву на позиция и уменьшаем длину токена на 1
            del_count = 0
            word = list(token.text)
            indexes = sorted(indexes)
            for idx in indexes:
                if len(word) == 1:
                    break
                try:
                    word.pop(idx-del_count)
                    del_count += 1
                except:
                    pass
            return (token.start, token.stop, ''.join(word), -del_count)
        elif mode == 'mixed':
            ins_count = 0
            del_count = 0
            word = list(token.text)
            # генерируем самое первое действие в слове
            idx2action = random.randint(0, 2)
            indexes = sorted(indexes)
            for idx in indexes:
                # вставляем букву на позиция и увеличиваем длину токена на 1, если ins
                # удаляем букву на позиция и уменьшаем длину токена на 1, если del
                # заменяем букву на позиции, если sub
                new_idx = idx+ins_count-del_count
                try:
                    if idx2action == 0:
                        symbol = word[new_idx]
                        word[new_idx] = key_errors[symbol][random.randint(0, len(key_errors[symbol]) - 1)]
                        idx2action += 1
                    elif idx2action == 1:
                        word.insert(new_idx, key_errors[symbol][random.randint(0, len(key_errors[symbol]) - 1)])
                        ins_count += 1
                        idx2action += 1
                    elif idx2action == 2:
                        word.pop(new_idx)
                        del_count += 1
                        idx2action = 0 
                except:
                    pass
            return (token.start, token.stop, ''.join(word), ins_count-del_count)

    # spoiled texts
    spoiled_text = list()
    pbar = tqdm(dataframe['text'], leave=True, position=0)
    for idx, sent, target1 in zip(
        range(len(dataframe['text'])), 
        dataframe['text'], 
        dataframe['0class']
    ):
        # get tokens of our text
        tokens = [data for data in list(tokenize(sent.lower()))]
        # just one word
        if len(tokens) == 1:
            spoiled_text.append(sent)
            continue
        
        # выбираем определенным методом наиболее важные слова
        if tok_importance in ['loss', 'lime', 'shap']:
            word2spoil_order = extract_essential_words(
                tokens, target1, tok_importance, tokens_type='razdel'
            )
        # выбираем случайным образом наиболее важные слова
        else:
            word2spoil_order = extract_random_words(tokens)

        sub_count = 0
        spoiled_tokens = list()
        for token in word2spoil_order:
            # get token and token's position
            token_len = token.stop - token.start
            # no way to change
            if token_len != 1 and sub_count < words2spoil: 
                # count our changes
                if mode2amount == 'percent':
                    sub_letter = max(1, int(token_len * sub_percent))
                elif mode2amount == 'amount':
                    sub_letter = max(1, sub_amount)
                # get indexes to change
                indexes = get_indexes2change(sub_letter, token_len)
                # go through indexes
                spoiled_word = make_token_change(indexes, token, mode2spoil)
                # increase our subs
                sub_count += 1
                spoiled_tokens.append(spoiled_word)
            # сделали нужное количество порч
            if sub_count >= words2spoil:
                break
        
        # заменяем исходные слов в тексте испорченными
        shift_in_sent = 0
        spoiled_sent = list(sent.lower())
        spoiled_tokens = sorted(spoiled_tokens, key=lambda x:x[0])
        for spoiled in spoiled_tokens:
            spoiled_start, spoiled_stop, spoiled_word, word_shift = spoiled
            spoiled_sent[spoiled_start + shift_in_sent:spoiled_stop + shift_in_sent] = spoiled_word
            shift_in_sent += word_shift
        spoiled_sent = ''.join(spoiled_sent)
        spoiled_text.append(spoiled_sent)
        
        pbar.update(1)
        pbar.set_description(f'Total processed: {idx + 1}')
        
    return spoiled_text

### Портим текст, вычисляем показатель use_score и accuracy

In [ ]:
def get_scores_char_spoiled_text(
        model: torch.nn.Module, dataframe: pd.DataFrame, 
        words2spoil: List[int], mode2amount: str, 
        sub_amount: List[int], sub_percent: List[float],
        spoil_modes: List[str], word_importances: List[str]
    ) -> Tuple[Dict[str, float], Dict[str, float], Dict[str, float], pd.DataFrame]:
    """
    dataframe: (pandas данные с текстом и метками)
    mode2amount: 'percent', 'amount' (по процентам или по количеству букв)
    words2spoil_amount: (количество слов для порчи)
    subs: (сколько букв испортить)
    subs_percent: (сколько букв в процентах от слова испортить)
    spoil_modes: 'mixed', 'ins', 'del', 'sub' (способы порчи текста)
    spoil_init: 'random', 'loss', 'lime' (тип выбора слов для порчи)
    """
    
    assert mode2amount in ['percent', 'amount']
    assert set(word_importances) <= {'loss', 'shap', 'lime', 'random', 'alti'}
    assert set(spoil_modes) <= {'mixed', 'ins', 'del', 'sub'}
    assert all(amount > 0 for amount in words2spoil)
    assert all(sub >= 1 for sub in sub_amount)
    assert all(sub >= 0.05 for sub in sub_percent)
    
    dan_scores = dict()
    bert_scores = dict()
    acc_scores = dict()
    
    subs = sub_percent if mode2amount == 'percent' else sub_amount

    for word_importance in word_importances:
        for sub in subs:
            for mode2spoil in spoil_modes:
                for words_amount in words2spoil:

                    col_name = f'{word_importance}_{words_amount}_{mode2spoil}_{mode2amount}_{sub}_CharSpoiledText'
                    
                    # генерируем состязательные примеры
                    spoiled_text = extract_spoiled_text_char_level(
                        dataframe, words2spoil=words_amount,
                        sub_amount=sub, sub_percent=sub, mode2amount=mode2amount, 
                        mode2spoil=mode2spoil, spoil_init=word_importance
                    )
                    
                    # сохраняем колонку со состязательными примерами
                    dataframe[col_name] = spoiled_text
                    
                    # считаем use score на основе представлений bert
                    _, use_result_char_bert = use_score(
                        dataframe['text'],
                        dataframe[col_name],
                        use_bert_encoder=True,
                        model=model
                    )
                    # считаем use score на основе dan кодировщика
                    _, use_result_char = use_score(
                        dataframe['text'],
                        dataframe[col_name]
                    )

                    sentidata = SentimentData(
                        dataframe=dataframe,
                        mode='test',
                        col_name=col_name
                    )

                    # замеряем качество состязательных примеров
                    spoiled_accuracy_char = calculate_accuracy(model, sentidata)
                    
                    # сохраняем результаты
                    dan_scores[col_name] = use_result_char
                    bert_scores[col_name] = use_result_char_bert
                    acc_scores[col_name] = spoiled_accuracy_char
                
    return dan_scores, bert_scores, acc_scores, dataframe

In [ ]:
mode2amount = 'amount'
sub_amount, sub_percent = [1], [0.05]
spoil_modes = ['ins', 'del', 'sub']
spoil_inits = ['shap']
words2spoil_amount = [1, 2]

dan_scores_char, bert_scores_char, acc_scores_char, adversial_examples_char = get_scores_char_spoiled_text(
    bert, adversial_examples_char,
    words2spoil_amount=words2spoil_amount,
    mode2amount=mode2amount,
    sub_amount=sub_amount, sub_percent=sub_percent,
    spoil_modes=spoil_modes, spoil_inits=spoil_inits
)

### Сохраняем все результаты

In [ ]:
# сохраняем состязательные примеры
adversial_examples_char.to_csv('adversial_examples_char.csv')

# создание pd.DataFrame с бъединенными данными
scores = [dan_scores_char, bert_scores_char, acc_scores_char]
names = ['dan_score', 'bert_score', 'acc_score']
dataframes = list()

# создаем список отдельных dataframe
for name, score in zip(names, scores):
    score_dct = {
        'modification': list(),
        name: list()
    }
    for key, val in score.items():
        score_dct['modification'].append(key)
        score_dct[name].append(val)
    dataframes.append(pd.DataFrame(score_dct))

# merge всех dataframe
init_dataframe = dataframes[0]
for i in range(1, len(dataframes)):
    init_dataframe = init_dataframe.merge(dataframes[i], how='left', on='modification')

init_dataframe['importance'] = init_dataframe['modification'].apply(lambda x: x.split('_')[0])
init_dataframe['modification'] = init_dataframe['modification'].apply(lambda x: '_'.join(x.split('_')[1:]))

# init_dataframe = init_dataframe.set_index('modification')

### Графики зависимостей символов

In [ ]:
orig_acc = 0.762
orig_dan = 1
orig_use = 1

def plot_char_results(method: str, modification2value: Dict[str, float]) -> None:
    
    # сохраняем mappings от (способ порчи, кол-во испорченных слов)
    # к полученному результату
    bert_method_to_res = dict()
    acc_method_to_res = dict()
    dan_method_to_res = dict()
    # есть ли в результатах данный метод оценки важности слова
    used_method = False
    # получаем имена всех модификаций
    modifications = modification2value.keys()
    for modification in modifications:
        modification_parts = modification.split('_')
        # получаем характеристики модификации
        importance = modification_parts[0]
        spoil_method = modification_parts[2]
        words_amount = modification_parts[1]
        sub = modification_parts[4]
        # если хотим визуализировать другой метод
        if importance != method:
            continue
        used_method = True
        # получаем и сохраняем результаты
        bert_score = bert_scores_char[modification]
        dan_score = dan_scores_char[modification]
        acc_score = acc_scores_char[modification]
        
        if bert_method_to_res.get((spoil_method, words_amount), None) is None:
            bert_method_to_res[(spoil_method, words_amount)] = [(orig_acc, 0)]
        if dan_method_to_res.get((spoil_method, words_amount), None) is None:
            dan_method_to_res[(spoil_method, words_amount)] = [(orig_dan, 0)]
        if acc_method_to_res.get((spoil_method, words_amount), None) is None:
            acc_method_to_res[(spoil_method, words_amount)] = [(orig_use, 0)]

        bert_method_to_res[(spoil_method, words_amount)].append((bert_score, sub))
        acc_method_to_res[(spoil_method, words_amount)].append((acc_score, sub))
        dan_method_to_res[(spoil_method, words_amount)].append((dan_score, sub))
    
    assert used_method
    
    names = ['accuracy', 'dan_sim', 'bert_sim']
    scores = [acc_method_to_res, dan_method_to_res, bert_method_to_res]
    _, axes = plt.subplots(3, 1, figsize=(20, 10))
    
    for idx, (name, mapping) in enumerate(zip(names, scores)):
        subs = None
        for key, value in mapping.items():
            if subs is None:
                subs = [sub for _, sub in value]
            results = [result for result, _ in value]

            axes[idx].plot(results, label=''.join(key))
        
        axes[idx].set_xlable('spoil chars amount')
        axes[idx].set_ylabel(name)
        axes[idx].set_title(f'{name} with {method} depending on spoil chars amount')
        axes[idx].set_xticklabel(subs)
        axes[idx].legend()
        axes[idx].grid(True)